### export the labels of the ontologies to json

require ARQ (Jena).Jena can be installed on mac using homebrew

In [65]:
import urllib.request
import json

### Trait Ontologie: TO

In [5]:
%%script bash

arq --results JSON --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/> 
        
SELECT ?label ?to 
WHERE {
    { ?to rdfs:label ?label }
    UNION
    { ?to oio:hasExactSynonym ?label }
    UNION
    { ?to oio:hasRelatedSynonym ?label }
    UNION
    { ?to oio:hasNarrowSynonym ?label }
    UNION
    { ?to oio:hasBroadSynonym ?label }
    
}
'>data/to.json

### Crop Ontology: CO

In [68]:
## list the CO IDs
url = "http://www.cropontology.org/get-ontologies"
r = urllib.request.urlopen(url)
data = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))

ontologies = data["300-499 Phenotype and Trait Ontology"]
ontologyList = []
for ontology in ontologies:
    if "CO_" in ontology["ontology_id"]:
        ontologyList.append({"id": ontology["ontology_id"] ,"name": ontology["ontology_name"]})

print(json.dumps(ontologyList, indent=2))
   

[
  {
    "name": "Banana",
    "id": "CO_325"
  },
  {
    "name": "Barley Trait Dictionary",
    "id": "CO_323"
  },
  {
    "name": "Brachiaria",
    "id": "CO_345"
  },
  {
    "name": "Cassava",
    "id": "CO_334"
  },
  {
    "name": "Castor bean",
    "id": "CO_347"
  },
  {
    "name": "Chickpea",
    "id": "CO_338"
  },
  {
    "name": "Common bean",
    "id": "CO_335"
  },
  {
    "name": "Cowpea",
    "id": "CO_340"
  },
  {
    "name": "Groundnut",
    "id": "CO_337"
  },
  {
    "name": "Lentil",
    "id": "CO_339"
  },
  {
    "name": "Maize",
    "id": "CO_322"
  },
  {
    "name": "Mungbean",
    "id": "CO_346"
  },
  {
    "name": "Oat",
    "id": "CO_350"
  },
  {
    "name": "Pearl millet",
    "id": "CO_327"
  },
  {
    "name": "Pigeonpea",
    "id": "CO_341"
  },
  {
    "name": "Potato",
    "id": "CO_330"
  },
  {
    "name": "Rice",
    "id": "CO_320"
  },
  {
    "name": "Sorghum",
    "id": "CO_324"
  },
  {
    "name": "Soybean",
    "id": "CO_336"
  },
  {


In [125]:
## which CO do you want to export? specify the CO ID
##CHANGE HERE
coID = "CO_343"
coROOT = "http://www.cropontology.org/rdf/"+coID+":ROOT"
#print(coROOT)

In [123]:
## get the CO file from the CO API and save it locally
url = "http://www.cropontology.org/ontology/"+coID+"/co/owl"
urllib.request.urlretrieve(url, "data/temp.owl")

('data/temp.owl', <http.client.HTTPMessage at 0x109305668>)

In [124]:
%%script bash -s "$coID" "$coROOT"

arq --results JSON --data data/temp.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        
SELECT ?label ?co 
WHERE {
    { ?co rdfs:label ?label . ?co rdfs:subClassOf* <'$2'> }
    UNION
    { ?co skos:altLabel ?label . ?co rdfs:subClassOf* <'$2'> }
    
}
'>data/$1.json